In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls

DNS-Challenge  gdrive  sample_data


In [ ]:
!ls /content/gdrive/'My Drive/Colab Notebooks'/

'2nd Cross_output_2ndtime.wav'	'2nd Cross_output.wav'	 cyclegan_1.ipynb
'2nd Cross_output_3rdtime.wav'	'2nd Cross Road 2.wav'	 dtln_model
'2nd Cross_output_4thtime.wav'	'Audio Recordings'	 DTLN_norm_40h.h5
'2nd Cross_output_5thtime.wav'	 checking		 Speech_denoising.ipynb


In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-bb2sq8lr
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-bb2sq8lr
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=e88f28751ada91a80310dedd9a89d26c7ba4d57e7b8c1b2d421c305cb6f9243b
  Stored in directory: /tmp/pip-ephem-wheel-cache-uvmkt_xi/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [ ]:
pip install librosa

In [ ]:
!git clone https://github.com/microsoft/DNS-Challenge DNS-Challenge

fatal: destination path 'DNS-Challenge' already exists and is not an empty directory.


In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.


In [ ]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 99 not upgraded.
Need to get 6,877 kB of archives.
After this operation, 16.4 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 2.11.0 [6,877 kB]
Fetched 6,877 kB in 1s (5,907 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [ ]:
!git lfs install

Git LFS initialized.


In [ ]:
import numpy as np
import tensorflow as tf


In [ ]:
model = tf.saved_model.load('/content/gdrive/My Drive/Colab Notebooks/dtln_model')
infer = model.signatures["serving_default"]

UnknownError: ignored

In [ ]:
model.variables

ListWrapper([<tf.Variable 'lstm/lstm_cell/kernel:0' shape=(257, 512) dtype=float32, numpy=
array([[-0.11317644,  0.3818724 , -0.01646251, ..., -0.08981922,
         0.05093318, -0.18282785],
       [-0.1635969 , -0.15192102,  0.08896536, ...,  0.20583394,
        -0.10045382,  0.03825117],
       [ 0.06326669,  0.01964286, -0.11296632, ...,  0.11689628,
        -0.02581113, -0.02544192],
       ...,
       [-0.59566116, -0.1762233 ,  0.33986688, ..., -0.26082623,
        -0.02111555, -0.7905301 ],
       [-0.58212626,  0.0774374 ,  0.22921044, ..., -0.54440236,
         0.33581236, -0.8273299 ],
       [-0.471445  ,  0.12248597,  0.20912328, ..., -0.33414504,
         0.4424216 , -1.1100721 ]], dtype=float32)>, <tf.Variable 'lstm/lstm_cell/recurrent_kernel:0' shape=(128, 512) dtype=float32, numpy=
array([[ 0.7048465 , -0.4869062 , -0.3100847 , ..., -0.20152466,
        -0.09938309, -0.4141071 ],
       [ 0.24651825,  0.8124956 ,  0.6645012 , ...,  0.37840608,
        -0.12192617,  0.15

In [ ]:
!pip install pysoundfile
import soundfile as sf

In [14]:
# load audio file at 16k fs (please change)
audio,fs = sf.read('/content/gdrive/My Drive/Colab Notebooks/checking/9(1)_output_4.wav')

RuntimeError: ignored

In [ ]:
audio,fs

(array([0.        , 0.        , 0.        , ..., 0.0022583 , 0.00204468,
        0.00137329]), 16000)

In [ ]:
audio.shape

(95365,)

In [ ]:
block_len = 512
block_shift = 128

In [ ]:
out_file = np.zeros((len(audio)))
in_buffer = np.zeros((block_len))
out_buffer = np.zeros((block_len))
# calculate number of blocks
num_blocks = (audio.shape[0] - (block_len-block_shift)) // block_shift

In [ ]:
num_blocks

742

In [ ]:
for idx in range(num_blocks):
    # shift values and write to buffer
    in_buffer[:-block_shift] = in_buffer[block_shift:]
    in_buffer[-block_shift:] = audio[idx*block_shift:(idx*block_shift)+block_shift]
    # create a batch dimension of one
    in_block = np.expand_dims(in_buffer, axis=0).astype('float32')
    # process one block
    out_block= infer(tf.constant(in_block))['conv1d_1']
    # shift values and write to buffer
    out_buffer[:-block_shift] = out_buffer[block_shift:]
    out_buffer[-block_shift:] = np.zeros((block_shift))
    out_buffer  += np.squeeze(out_block)
    # write block to output file
    out_file[idx*block_shift:(idx*block_shift)+block_shift] = out_buffer[:block_shift]

In [ ]:
sf.write('/content/gdrive/My Drive/Colab Notebooks/checking/9(1)_output_5.wav', out_file, fs) 